In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from glob import glob

In [2]:
def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())

    for key in list(data.keys()):
        if type(data[key]) == list:
            data[key] = tuple(data[key])
    return data

def dicts_equal(dict1, dict2):
    keys1 = tuple(sorted(list(dict1.keys())))
    keys2 = tuple(sorted(list(dict2.keys())))
    if keys1 != keys2:
        return False
    for key in keys1:
        if dict1[key] != dict2[key]:
            return False
    return True

In [3]:
MODELS = ['ISIBrno_model', 'CNN_model', 'RNN_model']
TRAIN_DATASETS = [['ptb_xl'], ['ptb_xl', 'ningbo'], ['ptb_xl', 'ningbo', 'georgia']]
EXP2_DATASETS = ['sph', 'code15']

In [4]:
results = sorted(glob('results/*.csv'))
print(len(results))
results = {log: pd.read_csv(log) for log in results}
print(len(results))
results = {key.removeprefix('results/').removesuffix('.csv'): val for key, val in results.items()}
results = {key: {'log': val, 'cfg': open_cfg(f'results/{key}.cfg')} for key, val in results.items() if os.path.isfile(f'results/{key}.cfg')}
print(len(results))
results = {key: val for key, val in results.items() if val['cfg']['ecg_encoder_model'] in MODELS}
print(len(results))

83
83
79
60


In [5]:
for model in MODELS:
    print()
    print(model)
    model_results = {key: val for key, val in results.items() if val['cfg']['ecg_encoder_model'] == model}
    if model == 'ISIBrno_model':
        model_results = {key: val for key, val in results.items() if val['cfg']['device'] == 'cuda:0'}
    print(len(model_results))

    for datasets in TRAIN_DATASETS:
        print()
        #print(datasets)
        ds_results = {key: val for key, val in model_results.items() if val['cfg']['train_datasets'] == tuple(datasets)}
        #print(len(ds_results))


        for train_ds in datasets:
            print(f'{train_ds} test metrics:', np.mean([res['cfg']['test_metrics'][train_ds]['mean_rocaucs'] for res in ds_results.values()]))

        
        for train_ds in datasets:
            print(f'{train_ds} zero-shot metrics:', np.mean([res['cfg']['zero_shot_test_metrics'][train_ds]['mean_rocaucs'] for res in ds_results.values()])) 


        for exp2_ds in EXP2_DATASETS:
            print(f'{exp2_ds} trained metrics:', np.mean([res['cfg']['exp2_metrics_trained'][exp2_ds]['mean_rocaucs'] for res in ds_results.values()]))


        for exp2_ds in EXP2_DATASETS:
            print(f'{exp2_ds} zero-shot metrics:', np.mean([res['cfg']['exp2_metrics_untrained'][exp2_ds]['mean_rocaucs'] for res in ds_results.values()]))
        #plt.figure(figsize=(30, 5))
        #for result in ds_results.values():
        #    plt.plot(result['log']['valid_mean_rocaucs'])
        #plt.grid()
        #plt.show()


ISIBrno_model
15

ptb_xl test metrics: 0.8417800415039324
ptb_xl zero-shot metrics: 0.6142071648952742
sph trained metrics: 0.8481176020422101
code15 trained metrics: 0.8698979854624547
sph zero-shot metrics: 0.7436166418893111
code15 zero-shot metrics: 0.7976568488029769

ptb_xl test metrics: 0.8225428132306085
ningbo test metrics: 0.843967740328598
ptb_xl zero-shot metrics: 0.6317682913149696
ningbo zero-shot metrics: 0.5754033675136482
sph trained metrics: 0.8835258896970821
code15 trained metrics: 0.8825568036654079
sph zero-shot metrics: 0.7293810904115545
code15 zero-shot metrics: 0.7550639351539534

ptb_xl test metrics: 0.8306654472220588
ningbo test metrics: 0.8254365998996006
georgia test metrics: 0.779711683251671
ptb_xl zero-shot metrics: 0.6201398610762267
ningbo zero-shot metrics: 0.5898010425127357
georgia zero-shot metrics: 0.5785996250657255
sph trained metrics: 0.9038137190591087
code15 trained metrics: 0.9303005988273595
sph zero-shot metrics: 0.6882117955586889
code